# SUDOKU

Un tablero $M$ es una matriz $[N]^{N\times N}$. Un conjunto de índices $B\subset[N]\times N$ es un **bloque** si todos los elementos $\{M_{i,j}: (i,j) \in B\}$ son diferentes. El tablero es un **sudoku** si $N=n^{2}$ es un cuadrado perfecto y además, cada fila, cada columna y cada cuadrante son bloques. Aquí, un cuadrante es un conjunto de la forma $q^{(i,j)}=\{(ni+a,nj+b): a,b \in [n]\}$ con $i,j \in \{0, 1, \ldots, n-1\}$, en otrasa palabras, los cuadrantes se obtienen al dividir el tablero $n^{2}$ por $n^{2}$ en subtableros de $n$ por $n$.

En el **juego del sudoku** recibimos como entrada un sudoku incompleto, es decir, un tablero que podría tener algunos casilleros vacíos, representados con un 0. Podemos escribir un **PLE** que complete el tablero $M$ introduciendo variables binarias ${x_{ijk}: i,j,k \in [N]}$, tal que $x_{ijk}=1$ si y solo si $M[i,j]=k$.

## 1. Escriba el conjunto lineal entero del sudoku
Asuma como conocido todos los bloques $B$ de $M$.

\begin{align}
& \underset{k\in[N]}{\sum}x_{ijk}=1, \quad \forall (i,j) \in [N]\times [N]\\
& \underset{(i,j)\in B}{\sum}x_{ijk}=1, \quad \forall k \in [N]\; \forall B\\
& x_{ijk}=1, \quad \quad  \quad \; \forall (i,j,k) \in [N]^{3}:M[i,j]=k\\
& x_{ijk}\in\{0,1\} \quad \quad \forall(i,j,k) \in [N]^{3}
\end{align}

## 2.  Obtener bloques
Cree una función que tome como entrada $n$ y retorne un array $B$ con todos los bloques de un tablero de dimensión $n^2$.

In [1]:
function get_blocks(n)
    N = n^2
    #bloques
    Blocks= []
    #cuadrantes
    for i in 0:n-1, j in 0:n-1
        q = [(n*i+a, n*j+b) for a in 1:n, b in 1:n]
        push!(Blocks,q)
    end
    #filas y columnas
    for i in 1:N
        r = [(i,j) for j in 1:N]
        c = [(j,i) for j in 1:N]
        push!(Blocks,r)
        push!(Blocks,c)
    end
    return Blocks
end

get_blocks (generic function with 1 method)

In [2]:
get_blocks(3)

27-element Array{Any,1}:
 Tuple{Int64,Int64}[(1, 1) (1, 2) (1, 3); (2, 1) (2, 2) (2, 3); (3, 1) (3, 2) (3, 3)]      
 Tuple{Int64,Int64}[(1, 4) (1, 5) (1, 6); (2, 4) (2, 5) (2, 6); (3, 4) (3, 5) (3, 6)]      
 Tuple{Int64,Int64}[(1, 7) (1, 8) (1, 9); (2, 7) (2, 8) (2, 9); (3, 7) (3, 8) (3, 9)]      
 Tuple{Int64,Int64}[(4, 1) (4, 2) (4, 3); (5, 1) (5, 2) (5, 3); (6, 1) (6, 2) (6, 3)]      
 Tuple{Int64,Int64}[(4, 4) (4, 5) (4, 6); (5, 4) (5, 5) (5, 6); (6, 4) (6, 5) (6, 6)]      
 Tuple{Int64,Int64}[(4, 7) (4, 8) (4, 9); (5, 7) (5, 8) (5, 9); (6, 7) (6, 8) (6, 9)]      
 Tuple{Int64,Int64}[(7, 1) (7, 2) (7, 3); (8, 1) (8, 2) (8, 3); (9, 1) (9, 2) (9, 3)]      
 Tuple{Int64,Int64}[(7, 4) (7, 5) (7, 6); (8, 4) (8, 5) (8, 6); (9, 4) (9, 5) (9, 6)]      
 Tuple{Int64,Int64}[(7, 7) (7, 8) (7, 9); (8, 7) (8, 8) (8, 9); (9, 7) (9, 8) (9, 9)]      
 Tuple{Int64,Int64}[(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9)]
 Tuple{Int64,Int64}[(1, 1), (2, 1), (3, 1), (4, 1), (5,

## 3. Programe el PLE

Programe la formulación del sudoku para una tablero $M$, de dimensiones $N=n^2$ y resuelva la siguiente instancia:

```julia
n = 3
N = n^2
M = [5 7 0 0 0 4 3 1 0; 
    1 3 0 5 0 0 0 2 0; 
    4 0 2 3 0 6 0 0 0; 
    9 0 7 0 6 0 1 0 0; 
    0 0 0 2 0 7 0 0 0; 
    0 0 3 0 8 0 7 0 4; 
    0 0 0 7 0 2 9 0 1; 
    0 4 0 0 0 1 0 7 8; 
    0 2 1 8 0 0 0 4 5];
```

La solución que debe obtener es

```julia
    [5 7 8 9 2 4 3 1 6; 
    1 3 6 5 7 8 4 2 9; 
    4 9 2 3 1 6 8 5 7; 
    9 5 7 4 6 3 1 8 2; 
    8 1 4 2 9 7 5 6 3; 
    2 6 3 1 8 5 7 9 4; 
    6 8 5 7 4 2 9 3 1; 
    3 4 9 6 5 1 2 7 8; 
    7 2 1 8 3 9 6 4 5]

```

In [3]:
using JuMP, Gurobi

In [13]:
function sudoku_model(M)
    #tamaño del tablero
    N = size(M)[1]
    
    #obtener n tal que N=n^2, si n no es entero la siguiente linea arrojará un error
    n = Integer(N^0.5) 

    #obtener bloques
    Blocks = get_blocks(n)
    
    #crear modelo
    model = Model()
    #setear solver
    set_optimizer(model, Gurobi.Optimizer)
    #setear atributos del solver
    set_optimizer_attributes(model, "Presolve"=>0, "OutputFlag"=>false)
    
    #crear variables
    @variable(model, x[1:N,1:N, 1:N], Bin)
    #fijamos variables 
    for i in 1:N, j in 1:N
        if M[i,j]!=0
            fix(x[i,j,M[i,j]], 1)
        end
    end

    #cada elemento del tablero puede tener un elemento
    @constraint(model, [i in 1:N, j in 1:N], sum(x[i,j,k] for k in 1:N)==1)

    #todos los elementos de un bloque deben ser distintos
    @constraint(model, [B in Blocks, k in 1:N], sum(x[i,j,k] for (i,j) in B) == 1)

    #resolver
    optimize!(model)
    #estado de término
    term_status = termination_status(model)
    is_optimal = term_status == MOI.OPTIMAL
        
    #imprimir solución si se alcanzo el óptimo
    if is_optimal
        M_sol = zeros(Int64, N,N)
        for i in 1:N, j in 1:N, k in 1:N
           if value(model[:x][i,j,k]) == 1
                M_sol[i,j] = k 
            end
        end

        return model, M_sol
    end
end

sudoku_model (generic function with 1 method)

In [7]:
n = 3
N = n^2
M = [5 7 0 0 0 4 3 1 0; 
    1 3 0 5 0 0 0 2 0; 
    4 0 2 3 0 6 0 0 0; 
    9 0 7 0 6 0 1 0 0; 
    0 0 0 2 0 7 0 0 0; 
    0 0 3 0 8 0 7 0 4; 
    0 0 0 7 0 2 9 0 1; 
    0 4 0 0 0 1 0 7 8; 
    0 2 1 8 0 0 0 4 5];

In [15]:
model, M_sol = sudoku_model(M);

Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [16]:
M_sol

9×9 Array{Int64,2}:
 5  7  8  9  2  4  3  1  6
 1  3  6  5  7  8  4  2  9
 4  9  2  3  1  6  8  5  7
 9  5  7  4  6  3  1  8  2
 8  1  4  2  9  7  5  6  3
 2  6  3  1  8  5  7  9  4
 6  8  5  7  4  2  9  3  1
 3  4  9  6  5  1  2  7  8
 7  2  1  8  3  9  6  4  5